In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import tensorflow_datasets as tfds
import os

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])

Name: /physical_device:GPU:0   Type: GPU


In [3]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

811008/807992 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 0us/step


'C:\\Users\\sunlin\\.keras\\datasets'

In [4]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [5]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [6]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [7]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: id=74, shape=(), dtype=string, numpy=b'While, every Grecian heart he tamed, and took'>, <tf.Tensor: id=75, shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: id=76, shape=(), dtype=string, numpy=b'that led off at the start must have been disabled out on the plain. I'>, <tf.Tensor: id=77, shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: id=78, shape=(), dtype=string, numpy=b"From slaughter'd Trojans, after Ocean's God">, <tf.Tensor: id=79, shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: id=80, shape=(), dtype=string, numpy=b'Around, a darksome trench; beyond, a fence'>, <tf.Tensor: id=81, shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: id=82, shape=(), dtype=string, numpy=b'of fleet Acamas chief of the Thracians. "Sons of Priam," said he, "how'>, <tf.Tensor: id=83, shape=(), dtype=int64, numpy=2>)


In [8]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

In [9]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [10]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'While, every Grecian heart he tamed, and took'


In [11]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[2976, 5831, 724, 3308, 11976, 6430, 12047, 15273]


In [12]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

all_encoded_data = all_labeled_data.map(encode_map_fn)

In [13]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

In [14]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: id=99547, shape=(16,), dtype=int64, numpy=
 array([ 2976,  5831,   724,  3308, 11976,  6430, 12047, 15273,     0,
            0,     0,     0,     0,     0,     0,     0], dtype=int64)>,
 <tf.Tensor: id=99551, shape=(), dtype=int64, numpy=0>)

In [15]:
vocab_size += 1

In [16]:
model = tf.keras.Sequential()

In [17]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

In [18]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [19]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [20]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
697/697 [==============================].0969 - accuracy: 0.37 - 21s 10s/step - loss: 1.0937 - accuracy: 0.38 - 21s 7s/step - loss: 1.0891 - accuracy: 0.3906 - 21s 5s/step - loss: 1.0855 - accuracy: 0.398 - 21s 4s/step - loss: 1.0802 - accuracy: 0.412 - 21s 3s/step - loss: 1.0777 - accuracy: 0.398 - 21s 3s/step - loss: 1.0762 - accuracy: 0.399 - 21s 3s/step - loss: 1.0733 - accuracy: 0.398 - 21s 2s/step - loss: 1.0694 - accuracy: 0.404 - 21s 2s/step - loss: 1.0647 - accuracy: 0.406 - 21s 2s/step - loss: 1.0630 - accuracy: 0.416 - 21s 2s/step - loss: 1.0579 - accuracy: 0.415 - 21s 2s/step - loss: 1.0546 - accuracy: 0.413 - 21s 2s/step - loss: 1.0516 - accuracy: 0.406 - 21s 1s/step - loss: 1.0487 - accuracy: 0.402 - 21s 1s/step - loss: 1.0441 - accuracy: 0.401 - 21s 1s/step - loss: 1.0393 - accuracy: 0.400 - 21s 1s/step - loss: 1.0350 - accuracy: 0.401 - 21s 1s/step - loss: 1.0297 - accuracy: 0.404 - 21s 1s/step - loss: 1.0226 - accuracy: 0.417 - 21s 1s/step - loss: 1.0185 - ac

In [22]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

79/79 [==============================].3248 - accuracy: 0.85 - 2s 899ms/step - loss: 0.3078 - accuracy: 0.859 - 2s 607ms/step - loss: 0.3115 - accuracy: 0.859 - 2s 460ms/step - loss: 0.3008 - accuracy: 0.867 - 2s 373ms/step - loss: 0.3021 - accuracy: 0.871 - 2s 314ms/step - loss: 0.3155 - accuracy: 0.869 - 2s 273ms/step - loss: 0.3252 - accuracy: 0.868 - 2s 242ms/step - loss: 0.3186 - accuracy: 0.863 - 2s 218ms/step - loss: 0.3238 - accuracy: 0.855 - 2s 198ms/step - loss: 0.3300 - accuracy: 0.854 - 2s 183ms/step - loss: 0.3504 - accuracy: 0.846 - 2s 170ms/step - loss: 0.3440 - accuracy: 0.849 - 2s 158ms/step - loss: 0.3445 - accuracy: 0.849 - 2s 149ms/step - loss: 0.3574 - accuracy: 0.848 - 2s 140ms/step - loss: 0.3523 - accuracy: 0.851 - 2s 133ms/step - loss: 0.3526 - accuracy: 0.853 - 2s 127ms/step - loss: 0.3504 - accuracy: 0.852 - 2s 121ms/step - loss: 0.3498 - accuracy: 0.849 - 2s 116ms/step - loss: 0.3457 - accuracy: 0.851 - 2s 111ms/step - loss: 0.3555 - accuracy: 0.847 - 2s 107